# Return type casting differences plum vs fastcore

In the example below you can see the differences between plum and fastcore wrt return type casting after function dispatch.

The summary is:

* Plum supports going from specific types to more generic types
* Fastcore **also** supports going from generic types to more specific types

I believe that for Transform the fastcore way makes more sense. The docs put it nicely:

> Type propagation - Whenever possible a transform tries to return data of the same type it received. Mainly used to maintain semantics of things like ArrayImage which is a thin wrapper of pytorch's Tensor. You can opt out of this behavior by adding ->None return type annotation.

Curious if you agree. But I'll continue implementing the fastcore way for now.

In [ ]:
from fastcore.transform import Transform
import plum

class FloatSubclass(float): pass

# Plum version - allows FloatSubclass -> float but not float -> FloatSubclass
@plum.dispatcher.dispatch
def plum_func_float_subclass(x) -> FloatSubclass: return x*2

@plum.dispatcher.dispatch
def plum_func_float(x) -> float: return x*2

# Fastcore version - allows float -> FloatSubclass but not FloatSubclass -> float  
@Transform
def fastcore_func_float_subclass(x) -> FloatSubclass: return x*2

@Transform
def fastcore_func_float(x) -> float: return x*2

# Test cases
x_float = 3.0
x_subclass = FloatSubclass(3.0)

In [ ]:
print("Plum tests:")
for fn,inp,name in [
    (plum_func_float_subclass, x_float, "float -> FloatSubclass"),
    (plum_func_float_subclass, x_subclass, "FloatSubclass -> FloatSubclass"),
    (plum_func_float, x_float, "float -> float"),
    (plum_func_float, x_subclass, "FloatSubclass -> float"),
]:
    try: res = type(fn(inp))
    except Exception as e: res = e
    print(f"{name}: {res}")

print("\nFastcore tests:")
for fn,inp,name in [
    (fastcore_func_float_subclass, x_float, "float -> FloatSubclass"),
    (fastcore_func_float_subclass, x_subclass, "FloatSubclass -> FloatSubclass"),
    (fastcore_func_float, x_float, "float -> float"),
    (fastcore_func_float, x_subclass, "FloatSubclass -> float"),
]:
    try: res = type(fn(inp))
    except Exception as e: res = e
    print(f"{name}: {res}")

Plum tests:
float -> FloatSubclass: Cannot convert `6.0` to `__main__.FloatSubclass`.
FloatSubclass -> FloatSubclass: Cannot convert `6.0` to `__main__.FloatSubclass`.
float -> float: <class 'float'>
FloatSubclass -> float: <class 'float'>

Fastcore tests:
float -> FloatSubclass: <class '__main__.FloatSubclass'>
FloatSubclass -> FloatSubclass: <class '__main__.FloatSubclass'>
float -> float: <class 'float'>
FloatSubclass -> float: <class 'float'>
